In [1]:
import pandas as pd
import numpy as np
import random
from pulp import *
import math

## Import and structure Data

### Helper functions to structure data into other formats


In [2]:
def get_fuel_cost(p):
    annual_fuel_cost = 0
    for k in p.keys():
        details = k.split('_')
        if details[2] in fuel_adj.keys():
            adj = fuel_adj[details[2]][details[6]]
        else:
            adj = 1
        if int(details[5]) <= 2028:
            annual_fuel_cost = annual_fuel_cost + consumption['_'.join(details[1:4]) + '_' + details[7]]*\
            fuel_cost[details[5] + '_' + details[7]]*veh_range['_'.join(details[1:4])] * p[k] * adj
    return annual_fuel_cost

def get_emissions_cost(p):
    annual_emissions = []
    dbg = []
    for y in years[0:5]:
        emit = 0        
        #x_dbg = 0
        for k in p.keys():
            details = k.split('_')
            if details[2] in fuel_adj.keys():
                adj = fuel_adj[details[2]][details[6]]
            else:
                adj = 1
            if details[5] == str(y):
                emit += consumption['_'.join(details[1:4]) + '_' + details[7]]*\
                fuel_emissions[details[5] + '_' + details[7]]*veh_range['_'.join(details[1:4])] * p[k] * adj
                if emit < 0:
                    print("negative emissions")
                
        annual_emissions.append(emit)
    return annual_emissions

In [3]:
def get_veh_cost(p):
    total_veh_cost = 0
    for k in p.keys():
        details = k.split('_')
        if int(details[3]) <= 2028:
            total_veh_cost = total_veh_cost + vehicle_cost['_'.join(details[1:5])] * p[k]
    return total_veh_cost

In [4]:
def verify_usage_demand(use_df):
    # verify demand constraints are met
#    use_df['Size'] = [x.split('_')[1] for x in use_df['ID']]
    use_df['capacity'] = [n*d for n, d in zip(use_df['Num_Vehicles'], use_df['Distance_per_vehicle(km)'])]
    verify_demand = use_df[['Year', 'Size', 'Distance_bucket','capacity']].groupby(['Year', 'Size', 'Distance_bucket'], as_index=False).sum()
    verify_demand = verify_demand.merge(demand, left_on=['Distance_bucket', 'Size', 'Year'], right_on=['Distance', 'Size', 'Year'])
    return verify_demand

In [50]:
# demand
def apply_demand_constraint(linear_program, availability, size):
    for y in years:
#    for y in [2028]:
#        for size in ['S1', 'S2', 'S3', 'S4']:        
        for demand_bucket in ['D1', 'D2', 'D3', 'D4']:
            df = availability[(availability['demand_year'] == y) & (availability['Size'] == size) & 
                              (availability['demand_distance'] == demand_bucket)]
            if df.shape[0] == 0:
                print(y, size, demand_bucket)
                return df
            specific_demand = df['Demand (km)'].iloc[0]
            constraint = []
            for _id, yr_range, u, s, f in zip(df['ID'], df['Yearly range (km)'], 
                                              df['usage_distance'], df['sell_year'], df['Fuel']):
                constraint.append(usage_variables['_'.join([_id, str(s), str(y), u, f])] * yr_range)
            linear_program += lpSum(x for x in constraint) >= specific_demand 
    return linear_program

In [6]:

def create_use_df(fuel_plan):
    year = []
    vid = []
    num_veh = []
    _fuel = []
    dbucket = []
    vdistance = []
    for y in years:
        for v in fuel_plan.keys():
            d = v.split('_')
            if d[5] == str(y):
                year.append(y)
                vid.append('_'.join(d[1:4]))
                num_veh.append(fuel_plan[v])
                _fuel.append(d[7])
                dbucket.append(d[6])
#                if d[2] in fuel_adj.keys():
#                    adj = fuel_adj[d[2]][d[6]]
#                else:
#                    adj = 1
#                vdistance.append(math.ceil(veh_range['_'.join(d[1:4])] * min(adj, 1)))
                vdistance.append(veh_range['_'.join(d[1:4])])
    use_df = pd.DataFrame({'Year': year,
                           'ID': vid,
                           'Num_Vehicles': num_veh,
                           'Type': 'Use',
                           'Fuel': _fuel,
                           'Distance_bucket': dbucket,
                           'Distance_per_vehicle(km)': vdistance}).sort_values('Year')#.to_csv('submission.csv')
    return use_df

In [7]:
def apply_limited_sales_constraint(linear_program, except_pct, other_pct):
    # limited sales
    for y in years:
        sold_vehicles = []
        in_service = []

        for v in purchase_objective.keys():
            details = v.split('_')
            if int(details[3]) == y:
                sold_vehicles.append(v)
            elif (int(details[3]) > y) & (int(details[2]) <= y):
                in_service.append(v)
        if y in except_pct.keys():
            linear_program += lpSum(purchase_variables[x] for x in sold_vehicles)*(1-except_pct[y])\
            <= lpSum(purchase_variables[x] for x in in_service)*except_pct[y]
        else:
            linear_program += lpSum(purchase_variables[x] for x in sold_vehicles)*(1-other_pct) <= lpSum(purchase_variables[x] for x in in_service)*other_pct
    return linear_program

In [8]:
def smart_round(x):
    if x > 0:
        return max(round(x), 1)
    else:
        return 0
    

In [9]:
def create_buy_sell_dfs(purchase_plan):
    # buy and sell df
    buy_year = []
    sell_year = []
    vid = []
    n_veh = []
    for p in purchase_plan.keys():
        details = p.split('_')
        buy_year.append(details[3])
        sell_year.append(details[4])
        n_veh.append(purchase_plan[p])
        vid.append('_'.join(details[1:4]))

    buy_df = pd.DataFrame({'Year': [int(y) for y in buy_year],
                           'ID': vid,
                           'Num_Vehicles': n_veh})

    sell_df = pd.DataFrame({'Year': [int(y) for y in sell_year],
                           'ID': vid,
                           'Num_Vehicles': n_veh})
    sell_df = sell_df[sell_df['Year'] < 2039]

    buy_df = buy_df.groupby(['Year', 'ID'], as_index=False).sum()
    sell_df = sell_df.groupby(['Year', 'ID'], as_index=False).sum()
    buy_df['Type'] = 'Buy'
    buy_df['Distance_per_vehicle(km)'] = 0
    sell_df['Type'] = 'Sell'
    sell_df['Distance_per_vehicle(km)'] = 0
    return buy_df, sell_df


In [10]:
def pct_of_inventory(buy_df, sell_df):
    buy_sell = buy_df[['Year', 'Num_Vehicles']].groupby(['Year'], as_index=False).sum().merge(sell_df[['Year', 'Num_Vehicles']].groupby(['Year'], as_index=False).sum(), on='Year') 
    running_total = 0
    running_column = []
    running_pct = []
    for b, s in zip(buy_sell['Num_Vehicles_x'], buy_sell['Num_Vehicles_y']):
        running_pct.append(s/(running_total+b))
        running_total = running_total + b - s # end of year
        running_column.append(running_total)
    buy_sell['running_total'] = running_column 
    buy_sell['pct'] = running_pct
    return buy_sell

In [11]:
def smart_pick(dictionary, k):
    if k in dictionary:
        return dictionary[k]
    else:
        return 0
    
def vehicle_running_total(buy_df, sell_df, use_df, dbg=None):
    dfs = []
    df = pd.DataFrame({'Year':years})
    
    for v, n in zip(buy_df['ID'], buy_df['Num_Vehicles']):
        
        bdf = buy_df[(buy_df['ID'] == v)].copy()              # this should just be 1
        sdf = sell_df[(sell_df['ID'] == v)].copy()           
        
        edf = use_df[(use_df['ID'] == v)].copy()
        edf = edf[['ID', 'Year', 'Num_Vehicles']].groupby(['ID', 'Year'], as_index=False).sum()
        
        buy_dict = {}
        sell_dict = {}
        expected_dict = {}
        for y, n in zip(bdf['Year'],bdf['Num_Vehicles']):
            buy_dict[y] = n
        for y, n in zip(sdf['Year'],sdf['Num_Vehicles']):
            sell_dict[y] = n
        for y, n in zip(edf['Year'],edf['Num_Vehicles']):
            expected_dict[y] = n

        running_total = 0
        buy = []
        sell = []
        available_beg = []
        available_end = []
        expected = []
        difference = []
        for y in years:
            
            buy_amt = smart_pick(buy_dict, y)
            buy.append(buy_amt)
            sell_amt = smart_pick(sell_dict, y)
            sell.append(sell_amt)
            expected_amt = smart_pick(expected_dict, y)
            expected.append(expected_amt)
            difference.append(running_total + buy_amt - expected_amt)
            available_beg.append(running_total + buy_amt)
            running_total += buy_amt - sell_amt
            available_end.append(running_total)
            
        if dbg:
            if dbg == v:
                return pd.DataFrame({'Year': years, v+'_beg': available_beg, v+'_end': available_end,
                                     'buy': buy, 'sell': sell, v+'_xpd':expected})
                
        df = df.merge(pd.DataFrame({'Year': years, v: difference}), on='Year')
        
        df.set_index('Year', inplace=True)
        
    return df.T
            #    buy_total = buy_df[['ID', 'Year', 'Num_Vehicles']].groupby(['ID', 'Year'], as_index=False).sum()
    

In [12]:
def available_check(available, purchase_plan, dbg=None):
    models = []
    needed_years = []
    needed_amt = []
    new_purchase_plan2 = purchase_plan.copy()
    for m, inventory in zip(available.index, available.to_numpy()):

        needed = [y for y, inv in zip(years, inventory) if inv < 0 ]
        amt = [abs(inv) for y, inv in zip(years, inventory) if inv < 0 ]
        if len(needed) > 0:
            print('need {}'.format(m))

            for y, a in zip(needed,amt):        
                models.append(m)
                needed_years.append(y)
                needed_amt.append(a)
                options = []
                for p in new_purchase_plan2.keys():
                    d = p.split('_')
                    d2 = m.split('_')
                    if '_'.join(d[1:4]) == '_'.join(d2[0:3]):
                        if int(d[4]) >= y:
                            options.append(p)
                            print("considering {} {} {} {}".format(p, d[3], d[4], y))

                if len(options) == 0:
                    print('this is a bigger problem')
                    print(y, a, m)
                new_vehicle = options[random.randint(0, len(options)-1)]
                new_purchase_plan2[new_vehicle] += a
                print(new_vehicle, new_purchase_plan2[new_vehicle], a)

    return pd.DataFrame({'ID': models, 'Years': needed_years, 'Amt': needed_amt}), new_purchase_plan2

In [13]:
def update_fuel_adj(verify_demand):
    # use miles driven instead of range for fuel
    fuel_adj = {}
    for size in range(4):
        fuel_adj['S' + str(size+1)] = {}
    for s, d, dmnd, cpcty in zip(verify_demand['Size'], verify_demand['Distance'], verify_demand['Demand (km)'], 
                                 verify_demand['capacity']):
        fuel_adj[s][d] = dmnd/cpcty
    return fuel_adj

In [14]:
path = 'c:\\users\\mark113\\desktop\\jupyter\\shell\\'
demand = pd.read_csv(path + 'dataset\\demand.csv')
fuels = pd.read_csv(path + 'dataset\\fuels.csv')
cost_profiles = pd.read_csv(path + 'dataset\\cost_profiles.csv')
vehicles = pd.read_csv(path + 'dataset\\vehicles.csv')
vehicles_fuels = pd.read_csv(path + 'dataset\\vehicles_fuels.csv')
carbon_df = pd.read_csv(path + 'dataset\\carbon_emissions.csv')

In [39]:
vehicles_s1 = vehicles[vehicles['Size'] == 'S1']

In [16]:
max_year = 2038
years = [y for y in range(2023, max_year+1)]
max_ownership = 10

In [17]:
# demand = demand[demand['Year'] <= max_year]
# vehicles = vehicles[vehicles['Year'] <= max_year]

In [18]:
emissions_limit = {}
for e, y in zip(carbon_df['Carbon emission CO2/kg'], carbon_df['Year']):
    emissions_limit[y] = e

In [19]:
# build some dictionaries for easier reference
fuel_adj = {}

consumption = {}
for v, f, c in zip(vehicles_fuels['ID'], vehicles_fuels['Fuel'], vehicles_fuels['Consumption (unit_fuel/km)']):
    consumption[v+'_'+f] = c
fuel_cost = {}
fuel_emissions = {}
for f, y, e, c in zip(fuels['Fuel'], fuels['Year'], fuels['Emissions (CO2/unit_fuel)'], fuels['Cost ($/unit_fuel)']):
    fuel_cost[str(y)+'_'+f] = c
    fuel_emissions[str(y)+'_'+f] = e     
veh_range = {}
for v, r in zip(vehicles['ID'], vehicles['Yearly range (km)']):
    veh_range[v] = r

In [20]:
vehicle_usage = []
for d in ['D1', 'D2', 'D3', 'D4']:
    for u in range(int(d[1])):
        for year in years:
            for y in range(max_ownership):
                df = vehicles[(vehicles['Distance']==d) & (vehicles['Year'] == year)].copy()
                if df.shape[0] > 0:
                    df['usage_distance'] = 'D' + str(u+1)
                    df['sell_year'] = df['Year'] + y
                    vehicle_usage.append(df.copy())
        
V = pd.concat(vehicle_usage)
V = V[V['sell_year'] <= (max_year+1)]
V.head(1000).tail()


,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance,usage_distance,sell_year
179,LNG_S3_2026,LNG,S3,2026,165150,73000,D4,D1,2031
67,Diesel_S1_2026,Diesel,S1,2026,92881,102000,D4,D1,2032
83,Diesel_S2_2026,Diesel,S2,2026,113643,106000,D4,D1,2032
99,Diesel_S4_2026,Diesel,S4,2026,165299,118000,D4,D1,2032
115,Diesel_S3_2026,Diesel,S3,2026,129140,73000,D4,D1,2032


In [21]:
vehicles_df = V.merge(vehicles_fuels[['ID', 'Fuel']], on='ID')

In [22]:
ownership_costs = [.06, .09, .12, .15, .18, .21, .24, .27, .3, .33]
accum_own_cost = [np.sum(ownership_costs[0:(x+1)]) for x in range(max_ownership)]
resale_cost = [.9, .8, .7, .6, .5, .4, .3, .3, .3, .3]

In [23]:
# 1. calculate vehcile cost
# 2. join demand to vehicles based on what's available for the demand year

vehicles_df.columns = ['ID', 'Vehicle', 'Size', 'vehicle_year', 'Cost ($)', 'Yearly range (km)', 'vehicle_distance', 
                       'usage_distance', 'sell_year', 'Fuel']
vehicle_cost = {}
for v, cost in zip(vehicles['ID'], vehicles['Cost ($)']):
    for yr_sold in range(max_ownership):
        if (int(v[-4:]) + yr_sold) > max_year:
            resale = 0
        else:
            resale = resale_cost[yr_sold]
        vehicle_cost[v + '_' + str(int(v[-4:])+yr_sold)] = cost *(1+accum_own_cost[yr_sold] - resale)
availability = demand.merge(vehicles_df, left_on=['Size', 'Distance'], right_on=['Size', 'usage_distance'])  
availability.columns = ['demand_year', 'Size', 'demand_distance', 'Demand (km)', 'ID', 'Vehicle', 'vehicle_year', 
                        'Cost ($)', 'Yearly range (km)', 'vehicle_distance', 'usage_distance', 'sell_year', 'Fuel']
availability = availability[(availability['vehicle_year'] <= availability['demand_year']) & 
                            ((availability['sell_year']) >= availability['demand_year'])] 

In [24]:
veh_fuel_combo = availability.copy()
veh_fuel_combo['distance_integer'] = [int(vd[1]) for vd in veh_fuel_combo['vehicle_distance']]
veh_fuel_combo = veh_fuel_combo.drop(['usage_distance', 'demand_distance', 'vehicle_distance', 'Demand (km)'], axis=1).drop_duplicates()
veh_fuel_combo.shape

(11900, 10)

In [25]:
# check for "dominance"
#  for the same buy, sell year, size, distance group
# one vehicle has better fuel cost, fuel emissions for every usage year and better
# vehicle cost than another, then it completely dominates that vehicle
dominated_id = []
dominated_sell_yr = []
dominated_fuel = []
for beg_yr in years:
    for end_yr in [beg_yr+1, beg_yr+2]:
        for db in ['D1', 'D2', 'D3', 'D4']:
            for sb in ['S1', 'S2', 'S3', 'S4']:
                vid_for_df = []
                sell_year_for_df = []
                fuel_for_df = []
                fuel_value = []
                e_value = []
                v_cost_for_df = []
                vd_df = []
                df = veh_fuel_combo[(veh_fuel_combo['vehicle_year']==beg_yr) & (veh_fuel_combo['sell_year']==end_yr) & 
                                    (veh_fuel_combo['Size']==sb)]
                for v in df['ID'].unique():
                    single_df = df[df['ID']==v]

                    for f in single_df['Fuel'].unique():
                        v_cost_for_df.append(vehicle_cost[v+'_'+str(end_yr)])
                        single_fuel_df = single_df[single_df['Fuel'] == f]
                        vid_for_df.append(v)
                        sell_year_for_df.append(end_yr)

                        fcost = sum([fuel_cost[str(dy)+'_'+f]*consumption[v+'_'+f]*veh_range[v]
                                     for dy in single_fuel_df['demand_year']])
                        ecost = sum([fuel_emissions[str(dy)+'_'+f]*consumption[v+'_'+f] for dy in single_fuel_df['demand_year']])
                        fuel_for_df.append(f)
                        fuel_value.append(fcost)
                        e_value.append(ecost)
                        vd_df.append(single_fuel_df['distance_integer'].iloc[0])


                comparison = pd.DataFrame({'ID': vid_for_df, 'fuel_value': fuel_value, 'emissions_value': e_value, 
                                           'sell_year': sell_year_for_df, 'vehicle_cost': v_cost_for_df,
                                          'idist': vd_df, 'fuel': fuel_for_df})
                comparison['total_cost'] = [x+y for x, y in zip(comparison['fuel_value'], comparison['vehicle_cost'])]

                for a in range(comparison.shape[0]-1):
                    for b in range(a+1, comparison.shape[0]):
                        v1, fv1, e1, s1, vc1, d1, f1, tc1 = comparison.iloc[a]
                        v2, fv2, e2, s2, vc2, d2, f2, tc2 = comparison.iloc[b]
                        if (e1 >= e2) & (tc1 >= tc2) & (d1 <= d2):
                            print("1 {} {} {} {}".format(v1, v2, f1, f2))
                            dominated_id.append(v1)
                            dominated_sell_yr.append(end_yr)
                            dominated_fuel.append(f1)
                        elif (e1 <= e2) & (tc1 <= tc2) & (d1 >= d2):
                            print("2 {} {} {} {}".format(v1, v2, f1, f2))
                            if (v2 == 'LNG_S4_2032') & (f1 == 'Electricity') & (f2 == 'LNG'):
                                ref_df = comparison
                            dominated_id.append(v2)
                            dominated_sell_yr.append(end_yr)
                            dominated_fuel.append(f2)
                            


1 Diesel_S1_2023 LNG_S1_2023 B20 LNG
1 Diesel_S1_2023 LNG_S1_2023 B20 BioLNG
1 Diesel_S1_2023 LNG_S1_2023 HVO BioLNG
1 Diesel_S2_2023 LNG_S2_2023 B20 LNG
1 Diesel_S2_2023 LNG_S2_2023 B20 BioLNG
1 Diesel_S2_2023 LNG_S2_2023 HVO BioLNG
1 Diesel_S3_2023 LNG_S3_2023 B20 LNG
1 Diesel_S3_2023 LNG_S3_2023 B20 BioLNG
1 Diesel_S3_2023 LNG_S3_2023 HVO BioLNG
1 Diesel_S4_2023 LNG_S4_2023 B20 LNG
1 Diesel_S4_2023 LNG_S4_2023 B20 BioLNG
1 Diesel_S4_2023 LNG_S4_2023 HVO BioLNG
1 Diesel_S1_2023 LNG_S1_2023 B20 LNG
1 Diesel_S1_2023 LNG_S1_2023 B20 BioLNG
1 Diesel_S1_2023 LNG_S1_2023 HVO BioLNG
1 Diesel_S2_2023 LNG_S2_2023 B20 LNG
1 Diesel_S2_2023 LNG_S2_2023 B20 BioLNG
1 Diesel_S2_2023 LNG_S2_2023 HVO BioLNG
1 Diesel_S3_2023 LNG_S3_2023 B20 LNG
1 Diesel_S3_2023 LNG_S3_2023 B20 BioLNG
1 Diesel_S3_2023 LNG_S3_2023 HVO BioLNG
1 Diesel_S4_2023 LNG_S4_2023 B20 LNG
1 Diesel_S4_2023 LNG_S4_2023 B20 BioLNG
1 Diesel_S4_2023 LNG_S4_2023 HVO BioLNG
1 Diesel_S1_2023 LNG_S1_2023 B20 LNG
1 Diesel_S1_2023 LNG_S1_202

1 Diesel_S1_2025 LNG_S1_2025 B20 LNG
1 Diesel_S1_2025 LNG_S1_2025 B20 BioLNG
1 Diesel_S1_2025 LNG_S1_2025 HVO BioLNG
1 Diesel_S2_2025 LNG_S2_2025 B20 LNG
1 Diesel_S2_2025 LNG_S2_2025 B20 BioLNG
1 Diesel_S2_2025 LNG_S2_2025 HVO BioLNG
1 Diesel_S3_2025 LNG_S3_2025 B20 LNG
1 Diesel_S3_2025 LNG_S3_2025 B20 BioLNG
1 Diesel_S3_2025 LNG_S3_2025 HVO BioLNG
1 Diesel_S4_2025 LNG_S4_2025 B20 LNG
1 Diesel_S4_2025 LNG_S4_2025 B20 BioLNG
1 Diesel_S4_2025 LNG_S4_2025 HVO BioLNG
1 Diesel_S1_2025 LNG_S1_2025 B20 LNG
1 Diesel_S1_2025 LNG_S1_2025 B20 BioLNG
1 Diesel_S1_2025 LNG_S1_2025 HVO BioLNG
1 Diesel_S2_2025 LNG_S2_2025 B20 LNG
1 Diesel_S2_2025 LNG_S2_2025 B20 BioLNG
1 Diesel_S2_2025 LNG_S2_2025 HVO BioLNG
1 Diesel_S3_2025 LNG_S3_2025 B20 LNG
1 Diesel_S3_2025 LNG_S3_2025 B20 BioLNG
1 Diesel_S3_2025 LNG_S3_2025 HVO BioLNG
1 Diesel_S4_2025 LNG_S4_2025 B20 LNG
1 Diesel_S4_2025 LNG_S4_2025 B20 BioLNG
1 Diesel_S4_2025 LNG_S4_2025 HVO BioLNG
1 Diesel_S1_2025 LNG_S1_2025 B20 LNG
1 Diesel_S1_2025 LNG_S1_202

1 Diesel_S2_2027 LNG_S2_2027 B20 LNG
1 Diesel_S2_2027 LNG_S2_2027 B20 BioLNG
1 Diesel_S2_2027 LNG_S2_2027 HVO BioLNG
1 Diesel_S3_2027 LNG_S3_2027 B20 LNG
1 Diesel_S3_2027 LNG_S3_2027 B20 BioLNG
1 Diesel_S3_2027 LNG_S3_2027 HVO BioLNG
1 Diesel_S4_2027 LNG_S4_2027 B20 LNG
1 Diesel_S4_2027 LNG_S4_2027 B20 BioLNG
1 Diesel_S4_2027 LNG_S4_2027 HVO BioLNG
1 Diesel_S1_2027 LNG_S1_2027 B20 LNG
1 Diesel_S1_2027 LNG_S1_2027 B20 BioLNG
1 Diesel_S1_2027 LNG_S1_2027 HVO BioLNG
1 Diesel_S2_2027 LNG_S2_2027 B20 LNG
1 Diesel_S2_2027 LNG_S2_2027 B20 BioLNG
1 Diesel_S2_2027 LNG_S2_2027 HVO BioLNG
1 Diesel_S3_2027 LNG_S3_2027 B20 LNG
1 Diesel_S3_2027 LNG_S3_2027 B20 BioLNG
1 Diesel_S3_2027 LNG_S3_2027 HVO BioLNG
1 Diesel_S4_2027 LNG_S4_2027 B20 LNG
1 Diesel_S4_2027 LNG_S4_2027 B20 BioLNG
1 Diesel_S4_2027 LNG_S4_2027 HVO BioLNG
1 Diesel_S1_2027 LNG_S1_2027 B20 LNG
1 Diesel_S1_2027 LNG_S1_2027 B20 BioLNG
1 Diesel_S1_2027 LNG_S1_2027 HVO BioLNG
1 Diesel_S2_2027 LNG_S2_2027 B20 LNG
1 Diesel_S2_2027 LNG_S2_202

1 Diesel_S2_2029 LNG_S2_2029 B20 LNG
1 Diesel_S2_2029 LNG_S2_2029 B20 BioLNG
1 Diesel_S2_2029 LNG_S2_2029 HVO BioLNG
1 Diesel_S3_2029 LNG_S3_2029 B20 LNG
1 Diesel_S3_2029 LNG_S3_2029 B20 BioLNG
1 Diesel_S3_2029 LNG_S3_2029 HVO BioLNG
1 Diesel_S4_2029 LNG_S4_2029 B20 LNG
1 Diesel_S4_2029 LNG_S4_2029 B20 BioLNG
1 Diesel_S4_2029 LNG_S4_2029 HVO BioLNG
1 Diesel_S1_2029 LNG_S1_2029 B20 LNG
1 Diesel_S1_2029 LNG_S1_2029 B20 BioLNG
1 Diesel_S1_2029 LNG_S1_2029 HVO BioLNG
1 Diesel_S2_2029 LNG_S2_2029 B20 LNG
1 Diesel_S2_2029 LNG_S2_2029 B20 BioLNG
1 Diesel_S2_2029 LNG_S2_2029 HVO BioLNG
1 Diesel_S3_2029 LNG_S3_2029 B20 LNG
1 Diesel_S3_2029 LNG_S3_2029 B20 BioLNG
1 Diesel_S3_2029 LNG_S3_2029 HVO BioLNG
1 Diesel_S4_2029 LNG_S4_2029 B20 LNG
1 Diesel_S4_2029 LNG_S4_2029 B20 BioLNG
1 Diesel_S4_2029 LNG_S4_2029 HVO BioLNG
1 Diesel_S1_2030 LNG_S1_2030 B20 LNG
1 Diesel_S1_2030 LNG_S1_2030 B20 BioLNG
1 Diesel_S1_2030 LNG_S1_2030 HVO BioLNG
1 Diesel_S2_2030 LNG_S2_2030 B20 LNG
1 Diesel_S2_2030 LNG_S2_203

1 Diesel_S1_2032 LNG_S1_2032 B20 LNG
1 Diesel_S1_2032 LNG_S1_2032 B20 BioLNG
1 Diesel_S1_2032 LNG_S1_2032 HVO BioLNG
2 BEV_S2_2032 Diesel_S2_2032 Electricity B20
2 BEV_S2_2032 Diesel_S2_2032 Electricity HVO
2 BEV_S2_2032 LNG_S2_2032 Electricity LNG
2 BEV_S2_2032 LNG_S2_2032 Electricity BioLNG
1 Diesel_S2_2032 LNG_S2_2032 B20 LNG
1 Diesel_S2_2032 LNG_S2_2032 B20 BioLNG
1 Diesel_S2_2032 LNG_S2_2032 HVO BioLNG
2 BEV_S3_2032 Diesel_S3_2032 Electricity B20
2 BEV_S3_2032 Diesel_S3_2032 Electricity HVO
2 BEV_S3_2032 LNG_S3_2032 Electricity BioLNG
1 Diesel_S3_2032 LNG_S3_2032 B20 LNG
1 Diesel_S3_2032 LNG_S3_2032 B20 BioLNG
1 Diesel_S3_2032 LNG_S3_2032 HVO BioLNG
2 BEV_S4_2032 Diesel_S4_2032 Electricity B20
2 BEV_S4_2032 Diesel_S4_2032 Electricity HVO
2 BEV_S4_2032 LNG_S4_2032 Electricity LNG
2 BEV_S4_2032 LNG_S4_2032 Electricity BioLNG
1 Diesel_S4_2032 LNG_S4_2032 B20 LNG
1 Diesel_S4_2032 LNG_S4_2032 B20 BioLNG
1 Diesel_S4_2032 LNG_S4_2032 HVO BioLNG
2 BEV_S1_2032 Diesel_S1_2032 Electricity B2

...

2 BEV_S2_2034 Diesel_S2_2034 Electricity B20
2 BEV_S2_2034 Diesel_S2_2034 Electricity HVO
2 BEV_S2_2034 LNG_S2_2034 Electricity LNG
2 BEV_S2_2034 LNG_S2_2034 Electricity BioLNG
1 Diesel_S2_2034 LNG_S2_2034 B20 LNG
1 Diesel_S2_2034 LNG_S2_2034 B20 BioLNG
1 Diesel_S2_2034 LNG_S2_2034 HVO BioLNG
2 BEV_S3_2034 Diesel_S3_2034 Electricity B20
2 BEV_S3_2034 Diesel_S3_2034 Electricity HVO
2 BEV_S3_2034 LNG_S3_2034 Electricity LNG
2 BEV_S3_2034 LNG_S3_2034 Electricity BioLNG
1 Diesel_S3_2034 LNG_S3_2034 B20 LNG
1 Diesel_S3_2034 LNG_S3_2034 B20 BioLNG
1 Diesel_S3_2034 LNG_S3_2034 HVO BioLNG
2 BEV_S4_2034 Diesel_S4_2034 Electricity B20
2 BEV_S4_2034 Diesel_S4_2034 Electricity HVO
2 BEV_S4_2034 LNG_S4_2034 Electricity LNG
2 BEV_S4_2034 LNG_S4_2034 Electricity BioLNG
1 Diesel_S4_2034 LNG_S4_2034 B20 LNG
1 Diesel_S4_2034 LNG_S4_2034 B20 BioLNG
1 Diesel_S4_2034 LNG_S4_2034 HVO BioLNG
2 BEV_S1_2034 Diesel_S1_2034 Electricity B20
2 BEV_S1_2034 Diesel_S1_2034 Electricity HVO
2 BEV_S1_2034 LNG_S1_2034 El

2 BEV_S2_2034 Diesel_S2_2034 Electricity B20
2 BEV_S2_2034 Diesel_S2_2034 Electricity HVO
2 BEV_S2_2034 LNG_S2_2034 Electricity LNG
2 BEV_S2_2034 LNG_S2_2034 Electricity BioLNG
1 Diesel_S2_2034 LNG_S2_2034 B20 LNG
1 Diesel_S2_2034 LNG_S2_2034 B20 BioLNG
1 Diesel_S2_2034 LNG_S2_2034 HVO BioLNG
2 BEV_S3_2034 Diesel_S3_2034 Electricity B20
2 BEV_S3_2034 Diesel_S3_2034 Electricity HVO
2 BEV_S3_2034 LNG_S3_2034 Electricity LNG
2 BEV_S3_2034 LNG_S3_2034 Electricity BioLNG
1 Diesel_S3_2034 LNG_S3_2034 B20 LNG
1 Diesel_S3_2034 LNG_S3_2034 B20 BioLNG
1 Diesel_S3_2034 LNG_S3_2034 HVO BioLNG
2 BEV_S4_2034 Diesel_S4_2034 Electricity B20
2 BEV_S4_2034 Diesel_S4_2034 Electricity HVO
2 BEV_S4_2034 LNG_S4_2034 Electricity LNG
2 BEV_S4_2034 LNG_S4_2034 Electricity BioLNG
1 Diesel_S4_2034 LNG_S4_2034 B20 LNG
1 Diesel_S4_2034 LNG_S4_2034 B20 BioLNG
1 Diesel_S4_2034 LNG_S4_2034 HVO BioLNG
2 BEV_S1_2035 Diesel_S1_2035 Electricity B20
2 BEV_S1_2035 Diesel_S1_2035 Electricity HVO
2 BEV_S1_2035 LNG_S1_2035 El

1 Diesel_S3_2035 Diesel_S3_2035 B20 HVO
1 Diesel_S3_2035 LNG_S3_2035 B20 LNG
1 Diesel_S3_2035 LNG_S3_2035 B20 BioLNG
1 Diesel_S3_2035 LNG_S3_2035 HVO BioLNG
2 BEV_S4_2035 Diesel_S4_2035 Electricity B20
2 BEV_S4_2035 Diesel_S4_2035 Electricity HVO
2 BEV_S4_2035 LNG_S4_2035 Electricity LNG
2 BEV_S4_2035 LNG_S4_2035 Electricity BioLNG
1 Diesel_S4_2035 Diesel_S4_2035 B20 HVO
1 Diesel_S4_2035 LNG_S4_2035 B20 LNG
1 Diesel_S4_2035 LNG_S4_2035 B20 BioLNG
1 Diesel_S4_2035 LNG_S4_2035 HVO BioLNG
2 BEV_S1_2035 Diesel_S1_2035 Electricity B20
2 BEV_S1_2035 Diesel_S1_2035 Electricity HVO
2 BEV_S1_2035 LNG_S1_2035 Electricity LNG
2 BEV_S1_2035 LNG_S1_2035 Electricity BioLNG
1 Diesel_S1_2035 Diesel_S1_2035 B20 HVO
1 Diesel_S1_2035 LNG_S1_2035 B20 LNG
1 Diesel_S1_2035 LNG_S1_2035 B20 BioLNG
1 Diesel_S1_2035 LNG_S1_2035 HVO BioLNG
2 BEV_S2_2035 Diesel_S2_2035 Electricity B20
2 BEV_S2_2035 Diesel_S2_2035 Electricity HVO
2 BEV_S2_2035 LNG_S2_2035 Electricity LNG
2 BEV_S2_2035 LNG_S2_2035 Electricity BioLN

1 Diesel_S4_2036 Diesel_S4_2036 B20 HVO
1 Diesel_S4_2036 LNG_S4_2036 B20 LNG
1 Diesel_S4_2036 LNG_S4_2036 B20 BioLNG
1 Diesel_S4_2036 LNG_S4_2036 HVO BioLNG
2 BEV_S1_2036 Diesel_S1_2036 Electricity B20
2 BEV_S1_2036 Diesel_S1_2036 Electricity HVO
2 BEV_S1_2036 LNG_S1_2036 Electricity LNG
2 BEV_S1_2036 LNG_S1_2036 Electricity BioLNG
1 Diesel_S1_2036 Diesel_S1_2036 B20 HVO
1 Diesel_S1_2036 LNG_S1_2036 B20 LNG
1 Diesel_S1_2036 LNG_S1_2036 B20 BioLNG
1 Diesel_S1_2036 LNG_S1_2036 HVO BioLNG
2 BEV_S2_2036 Diesel_S2_2036 Electricity B20
2 BEV_S2_2036 Diesel_S2_2036 Electricity HVO
2 BEV_S2_2036 LNG_S2_2036 Electricity LNG
2 BEV_S2_2036 LNG_S2_2036 Electricity BioLNG
1 Diesel_S2_2036 Diesel_S2_2036 B20 HVO
1 Diesel_S2_2036 LNG_S2_2036 B20 LNG
1 Diesel_S2_2036 LNG_S2_2036 B20 BioLNG
1 Diesel_S2_2036 LNG_S2_2036 HVO BioLNG
2 BEV_S3_2036 Diesel_S3_2036 Electricity B20
2 BEV_S3_2036 Diesel_S3_2036 Electricity HVO
2 BEV_S3_2036 LNG_S3_2036 Electricity LNG
2 BEV_S3_2036 LNG_S3_2036 Electricity BioLN

2 BEV_S2_2037 Diesel_S2_2037 Electricity B20
2 BEV_S2_2037 Diesel_S2_2037 Electricity HVO
2 BEV_S2_2037 LNG_S2_2037 Electricity LNG
2 BEV_S2_2037 LNG_S2_2037 Electricity BioLNG
1 Diesel_S2_2037 Diesel_S2_2037 B20 HVO
2 Diesel_S2_2037 LNG_S2_2037 HVO LNG
2 BEV_S3_2037 LNG_S3_2037 Electricity LNG
2 BEV_S3_2037 LNG_S3_2037 Electricity BioLNG
1 Diesel_S3_2037 Diesel_S3_2037 B20 HVO
2 Diesel_S3_2037 LNG_S3_2037 HVO LNG
2 BEV_S4_2037 Diesel_S4_2037 Electricity B20
2 BEV_S4_2037 LNG_S4_2037 Electricity LNG
2 BEV_S4_2037 LNG_S4_2037 Electricity BioLNG
1 Diesel_S4_2037 Diesel_S4_2037 B20 HVO
2 Diesel_S4_2037 LNG_S4_2037 HVO LNG
2 BEV_S1_2037 Diesel_S1_2037 Electricity B20
2 BEV_S1_2037 Diesel_S1_2037 Electricity HVO
2 BEV_S1_2037 LNG_S1_2037 Electricity LNG
2 BEV_S1_2037 LNG_S1_2037 Electricity BioLNG
1 Diesel_S1_2037 Diesel_S1_2037 B20 HVO
1 Diesel_S1_2037 LNG_S1_2037 B20 LNG
1 Diesel_S1_2037 LNG_S1_2037 B20 BioLNG
1 Diesel_S1_2037 LNG_S1_2037 HVO BioLNG
2 BEV_S2_2037 Diesel_S2_2037 Electricit

In [26]:
dominated_df = pd.DataFrame({'ID': dominated_id, 'sell_year': dominated_sell_yr, 'Fuel': dominated_fuel})
dominated_df['dominated'] = 1
dominant_availability = availability.merge(dominated_df.drop_duplicates(), how='left', on=['ID', 'sell_year', 'Fuel'])
print(dominant_availability.shape)
dominant_availability = dominant_availability[dominant_availability['dominated'].isna()]
print(dominant_availability.shape)

(43688, 14)
(40388, 14)


In [40]:
# create usage variable names
# create purchase variable names
# group usage variables by purchase variables

usage_variable_names = []
purchase_objective = {}
fuel_consumption_objective = {}

for v in vehicles_s1['ID']:
#for v in vehicles['ID']:
    df_v = dominant_availability[dominant_availability['ID'] == v]

    for s in df_v['sell_year'].unique():
        df = df_v[df_v['sell_year'] == s]

        for dy in df['demand_year'].unique():
            
            df_y = df[df['demand_year'] == dy]
            purchase_group =\
            ['_'.join([v, str(s), str(dy), ud, f]) for ud, f in zip(df_y['usage_distance'], df_y['Fuel'])]
            usage_variable_names = usage_variable_names + purchase_group
            if v+'_'+str(s) in purchase_objective.keys():
                purchase_objective[v+'_'+str(s)].append(purchase_group)
            else:
                purchase_objective[v+'_'+str(s)] = [purchase_group]
    for d in df_v['demand_year'].unique():
        for f in df_v['Fuel'].unique():
            df = df_v[(df_v['demand_year'] == d) & (df_v['Fuel'] == f)]
            fuel_consumption_objective[v+'_'+str(d)+'_'+f] =\
            ['_'.join([v, str(s), str(d), u, f])  for s, u in zip(df['sell_year'], df['usage_distance'])]

len(usage_variable_names)            

10082

In [30]:
demand_s1 = demand[demand['Size']=='S1']

### Run Optimizaton 

In [44]:
total_cost = LpProblem("Shell", LpMinimize)
usage_variables = pulp.LpVariable.dicts("usage", (_id for _id in usage_variable_names ), lowBound=0, cat="Integer")
purchase_variables = pulp.LpVariable.dicts("purchase", (_id for _id in purchase_objective.keys()) , lowBound=0, cat="Integer")

In [45]:
# fuel cost
objective = []
ct = 0
emissions_constraint = []
for k in fuel_consumption_objective.keys():
    details = k.split('_') # vehicle, size, purchase year, usage year, fuel
    usage_objective = [usage_variables[v]*fuel_cost[details[3]+'_'+details[4]]*\
                       consumption['_'.join(details[0:3])+'_'+details[4]]*\
                       veh_range['_'.join(details[0:3])]  for v in fuel_consumption_objective[k]]
    objective = objective + usage_objective

In [47]:
# Full ownership
objective = objective + [purchase_variables[v]*vehicle_cost[v]  for v in list(purchase_objective.keys())]

total_cost += lpSum(x for x in objective)

# the amount used must be less than the amount purchased
for v in purchase_objective.keys():
    for by_demand_year in purchase_objective[v]:
        total_cost += lpSum(usage_variables[x] for x in by_demand_year) <= purchase_variables[v]

### emissions constraint

In [48]:

for y in years:
    
    annual_emissions = []
    emissions_id = {}
    consumption_id = {}
    veh_range_id = {}

    for u in usage_variables.keys():
        d = u.split('_')
        if d[4] == str(y):
            annual_emissions.append(u)
            emissions_id[u] = d[4]+'_'+d[6]
            consumption_id[u] = '_'.join(d[0:3])+'_'+d[6]
            veh_range_id[u] = '_'.join(d[0:3])

    total_cost += lpSum(usage_variables[v]*fuel_emissions[emissions_id[v]]*\
                        consumption[consumption_id[v]]*\
                        veh_range[veh_range_id[v]]  for v in annual_emissions) <= emissions_limit[y]

In [51]:
total_cost = apply_demand_constraint(total_cost, dominant_availability, 'S1')
total_cost = apply_limited_sales_constraint(total_cost, {2033: .2, 2031: .2}, .2)

In [55]:
print(total_cost.solve(PULP_CBC_CMD(timeLimit=240, keepFiles=False, mip=True, warmStart=False)))
#print(total_cost.solve(PULP_CBC_CMD(timeLimit=600, keepFiles=False, mip=False, warmStart=False)))
#print(total_cost.solve())
print(LpStatus[total_cost.status])

1
Optimal


### Verification

In [56]:
# round up for purchase and fuel plans
purchase_plan = {}
fuel_plan = {}

for v in total_cost.variables():
    if v.name != '__dummy':
        if v.varValue:
            if v.varValue > 0:
                d = v.name.split('_')
                if d[0] == 'purchase':
                    purchase_plan[v.name] = smart_round(v.varValue)
#                    purchase_plan[v.name] = v.varValue
                else:
                    fuel_plan[v.name] = smart_round(v.varValue)
#                    fuel_plan[v.name] = v.varValue

print(100 - 70*(get_fuel_cost(fuel_plan)-100000 + get_veh_cost(purchase_plan))/ 65000000)

76.41335036460234


In [51]:
buy_df, sell_df = create_buy_sell_dfs(purchase_plan)

In [52]:
# verify that demand has been met
use_df = create_use_df(fuel_plan)

In [54]:
submission = pd.concat([buy_df, use_df, sell_df])
submission = submission[submission['Num_Vehicles'] > 0]
submission['Year'] = submission['Year'].astype(int)
submission['Num_Vehicles'] = submission['Num_Vehicles'].astype(int)
submission['Distance_per_vehicle(km)'] = submission['Distance_per_vehicle(km)'].astype(float)

In [55]:
submission[['Year', 'ID', 'Num_Vehicles', 'Type', 'Fuel', 'Distance_bucket', 'Distance_per_vehicle(km)']].to_csv('solution72.csv', index=False)

In [282]:
# verify demand (include size)
use_df['Size'] = [s.split('_')[1] for s in use_df['ID']]
use_df = use_df[['Year', 'ID', 'Num_Vehicles', 'Size','Distance_bucket', 'Distance_per_vehicle(km)', 'Fuel']].\
groupby(['Year', 'ID', 'Distance_bucket', 'Size',
         'Distance_per_vehicle(km)',
         'Fuel'], as_index=False).sum()

verify_demand = verify_usage_demand(use_df)
verify_demand['demand_check'] = [c-d for c, d in zip(verify_demand['capacity'], verify_demand['Demand (km)'])]
v_df = verify_demand[verify_demand['demand_check'] < 0]
v_df.head()

,Year,Size,Distance_bucket,capacity,Distance,Demand (km),demand_check
1,2023,S1,D2,2550000,D2,2597094,-47094
2,2023,S1,D3,3264000,D3,3292011,-28011
3,2023,S1,D4,408000,D4,414315,-6315
5,2023,S2,D2,1378000,D2,1383196,-5196
6,2023,S2,D3,742000,D3,778008,-36008


In [40]:
# combine consumption and fuel cost
# get most expensive vehicle, fuel combination by year, size, distance
fuel_cost_combined = []
veh_name = []
usage_year = []
size = []
distance = []
fuel_for_df = []
for y in years:
    for f in fuel_plan.keys():
        d = f.split('_')
        v = '_'.join(d[1:4])
        veh_name.append(v)
        usage_year.append(d[5])
        size.append(d[2])
        distance.append(d[6])
        fuel_for_df.append(d[7])
        fuel_cost_combined.append(fuel_cost[d[5]+'_'+d[7]]*consumption[v+'_'+d[7]])
fuel_cost_df = pd.DataFrame({'ID': veh_name, 'usage_yr': usage_year, 'total_fuel_cost': fuel_cost_combined,
                            'Size':size, 'Distance':distance, 'Fuel': fuel_for_df})
max_cost = fuel_cost_df[['usage_yr', 'total_fuel_cost', 'Size', 'Distance']].groupby(['usage_yr', 'Size', 'Distance'], as_index=False).max()


In [41]:
df = fuel_cost_df.merge(max_cost, how='left', on=['usage_yr','Size', 'Distance'])
most_expensive = df[df['total_fuel_cost_x']==df['total_fuel_cost_y']].groupby(['usage_yr', 'Size', 'Distance'], as_index=False).first()#drop_duplicates()
most_expensive['usage_yr'] = [int(x) for x in most_expensive['usage_yr']]
most_expensive = most_expensive.merge(demand, left_on=['usage_yr', 'Size', 'Distance'], right_on=['Year', 'Size', 'Distance'])

In [42]:
most_expensive = most_expensive[['ID', 'usage_yr', 'Size', 'Distance','Demand (km)', 'Fuel']].\
merge(use_df[['ID', 'Num_Vehicles', 'Distance_bucket', 'Size', 'Year', 'Fuel']], how='right', 
      left_on=['ID', 'usage_yr', 'Size', 'Distance', 'Fuel'], right_on=['ID', 'Year', 'Size', 'Distance_bucket', 'Fuel'])


In [43]:
# join back to use_df
# if vehicle, fuel combo is most expensive for each size, distance and year
# use reduced mileage

fuel_adj = {}
xp_id = {}
xp_fuel = {}
xpnsv_range = []
not_xpnsv_range = []
sizes_for_df = []
years_for_df = []
xpnsv_id = []
xpnsv_fuel = []

distances_for_df = []
ratio_for_df = []
for y in years:
    fuel_adj[y] = {}
    xp_id[y] = {}
    xp_fuel[y] = {}
    for distance in ['D1', 'D2', 'D3', 'D4']:
        fuel_adj[y][distance] = {}
        xp_id[y][distance] = {}
        xp_fuel[y][distance] = {}
        for size in ['S1', 'S2', 'S3', 'S4']:
            df = most_expensive[(most_expensive['Size'] == size) & 
                                (most_expensive['Distance_bucket'] == distance) & 
                                (most_expensive['Year'] == y)]
            df_not_xpnsv = df[df['Distance'].isna()]
            df_xpnsv = df[~df['Distance'].isna()]
            if df_not_xpnsv.shape[0] > 0:
                not_expensive = sum([n*veh_range[v] for n, v in zip(df_not_xpnsv['Num_Vehicles'], df_not_xpnsv['ID'])])
            else:
                not_expensive = 0
            xpnsv_id.append(df_xpnsv['ID'].iloc[0])
            xpnsv_fuel.append(df_xpnsv['Fuel'].iloc[0])
            not_xpnsv_range.append(not_expensive)
            expensive = sum([n*veh_range[v] for n, v in zip(df_xpnsv['Num_Vehicles'], df_xpnsv['ID'])])
            xpnsv_range.append(expensive)
            sizes_for_df.append(size)
            years_for_df.append(y)
            distances_for_df.append(distance)
            dmnd = df_xpnsv['Demand (km)'].iloc[0]
            fuel_adj[y][distance][size] = (dmnd - not_expensive)/expensive
            
            ratio_for_df.append((dmnd - not_expensive)/expensive)
            xp_id[y][distance][size] = df_xpnsv['ID'].iloc[0]
            xp_fuel[y][distance][size]  = df_xpnsv['Fuel'].iloc[0]
mileage_by_cost = pd.DataFrame({'xp_id':xpnsv_id, 'Size': sizes_for_df, 'Distance': distances_for_df, 'Year': years_for_df, 
                                'xp_range': xpnsv_range, 'not_xp_range': not_xpnsv_range, 'Fuel': xpnsv_fuel, 
                                'xp_ratio': ratio_for_df})

In [44]:
mileage_by_cost['total_range'] = [x+nx for x, nx in zip(mileage_by_cost['xp_range'], mileage_by_cost['not_xp_range'])]
mileage_by_cost = mileage_by_cost.merge(demand, on=['Size', 'Year', 'Distance'])
mileage_by_cost['ratio'] = [(d-nxr)/xr for d, xr, nxr in zip(mileage_by_cost['Demand (km)'], mileage_by_cost['xp_range'], 
                                                             mileage_by_cost['not_xp_range'])]

In [45]:
new_use_df = use_df.copy()
new_use_df['Distance_per_vehicle(km)'] = [math.ceil(fuel_adj[y][db][s]*dpv) if (v == xp_id[y][db][s]) & (f == xp_fuel[y][db][s]) else dpv 
                                          for v, y, db, s, dpv, f in zip(use_df['ID'], use_df['Year'],  use_df['Distance_bucket'], 
                                                                      use_df['Size'], use_df['Distance_per_vehicle(km)'],
                                                                      use_df['Fuel'])]

In [46]:
new_use_df['Type'] = 'Use'

In [47]:
submission = pd.concat([buy_df, new_use_df, sell_df])
submission = submission[submission['Num_Vehicles'] > 0]
submission['Year'] = submission['Year'].astype(int)
submission['Num_Vehicles'] = submission['Num_Vehicles'].astype(int)
submission['Distance_per_vehicle(km)'] = submission['Distance_per_vehicle(km)'].astype(float)

In [48]:
submission[['Year', 'ID', 'Num_Vehicles', 'Type', 'Fuel', 'Distance_bucket', 'Distance_per_vehicle(km)']].to_csv('solution69.csv', index=False)

In [794]:
# verify demand (include size)
new_use_df['Size'] = [s.split('_')[1] for s in use_df['ID']]
new_use_df = new_use_df[['Year', 'ID', 'Num_Vehicles', 'Size','Distance_bucket', 'Distance_per_vehicle(km)', 'Fuel']].\
groupby(['Year', 'ID', 'Distance_bucket', 'Size',
         'Distance_per_vehicle(km)',
         'Fuel'], as_index=False).sum()

verify_demand = verify_usage_demand(new_use_df)
verify_demand['demand_check'] = [c-d for c, d in zip(verify_demand['capacity'], verify_demand['Demand (km)'])]
v_df = verify_demand[verify_demand['demand_check'] < 0]
v_df.head()

,Year,Size,Distance_bucket,capacity,Distance,Demand (km),demand_check


In [718]:
fuel_adj = update_fuel_adj(verify_demand)
print(100 - 70*(get_fuel_cost(fuel_plan) + get_veh_cost(purchase_plan))/ 65000000)

16.967673756558327


In [719]:
buy_df, sell_df = create_buy_sell_dfs(purchase_plan)
running_df = pct_of_inventory(buy_df, sell_df)
running_df

,Year,Num_Vehicles_x,Num_Vehicles_y,running_total,pct
0,2023,199,35,164,0.175879
1,2024,39,36,167,0.177340
2,2025,41,37,171,0.177885
3,2026,46,39,178,0.179724
4,2027,40,39,179,0.178899
5,2028,50,41,188,0.179039
6,2029,47,42,193,0.178723
7,2030,43,42,194,0.177966
8,2031,50,43,201,0.176230
9,2032,49,44,206,0.176000


In [284]:
fuels.merge(fuels_vehicles, on='ID')

NameError: name 'fuels_vehicles' is not defined

In [ ]:
vehicles_fuels['']

In [313]:
def increase_fuel_purchase_plans(fuel_plan, purchase_plan, v_df):

    # adjust for rounding fuel off
    # increases purchase plan and fuel plan
    # one at a time
    
    random.seed(192)
    new_fuel_plan = fuel_plan.copy()
    new_purchase_plan = purchase_plan.copy()
    for y, s, di, dfct in zip(v_df['Year'], v_df['Size'],  v_df['Distance'], v_df['demand_check']):

        buy_yr = random.randint(2023, 2039)
        sell_yr = buy_yr + random.randint(0, 9)
        while (buy_yr > y) or (sell_yr < y):
            buy_yr = random.randint(2023, 2039)
            sell_yr = buy_yr + random.randint(0, 9)

        options = []
        for p in fuel_plan.keys():
            d = p.split('_')
            if d[2] == s and d[5] == str(y) and d[6] == di:
                options.append(p)    

        selection = options[random.randint(0, len(options)-1)]
        d2 = selection.split('_')
        additional_capacity = veh_range['_'.join(d2[1:4])]
        if additional_capacity + dfct < 0:
            n_additions = 2
        else:
            n_additions = 1
        if selection in new_fuel_plan.keys():
            new_fuel_plan[selection] += n_additions
        else:
            new_fuel_plan[selection] = n_additions

        purchase_key = 'purchase_'+'_'.join(d2[1:5])
        if purchase_key in new_purchase_plan.keys():
            new_purchase_plan[purchase_key] += n_additions
        else:
            new_purchase_plan[purchase_key] = n_additions
        print("buying {}, for {} {} {} {} {}".format(purchase_key, y, s, di, dfct, additional_capacity))
    #    if purchase_key =='purchase_BEV_S2_2024_2031':
    #        break
    return new_fuel_plan, new_purchase_plan
new_fuel_plan, new_purchase_plan = increase_fuel_purchase_plans(fuel_plan, purchase_plan, v_df)
new_use_df = create_use_df(new_fuel_plan)

buying purchase_LNG_S1_2023_2025, for 2023 S1 D2 -47094 102000
buying purchase_LNG_S1_2023_2031, for 2023 S1 D4 -6315 102000
buying purchase_LNG_S2_2023_2024, for 2023 S2 D1 -41694 106000
buying purchase_LNG_S2_2023_2027, for 2023 S2 D3 -36008 106000
buying purchase_LNG_S2_2023_2031, for 2023 S2 D4 -27677 106000
buying purchase_LNG_S3_2023_2023, for 2023 S3 D2 -22901 73000
buying purchase_LNG_S4_2023_2024, for 2023 S4 D2 -46717 118000
buying purchase_LNG_S4_2023_2024, for 2023 S4 D3 -899 118000
buying purchase_BEV_S1_2024_2026, for 2024 S1 D1 -61242 102000
buying purchase_LNG_S1_2023_2025, for 2024 S1 D4 -11981 102000
buying purchase_LNG_S2_2023_2026, for 2024 S2 D3 -65076 106000
buying purchase_LNG_S2_2023_2026, for 2024 S2 D4 -27712 106000
buying purchase_LNG_S3_2023_2024, for 2024 S3 D1 -49440 73000
buying purchase_LNG_S3_2024_2032, for 2024 S3 D3 -7375 73000
buying purchase_LNG_S4_2023_2024, for 2024 S4 D3 -4186 118000
buying purchase_BEV_S1_2025_2027, for 2025 S1 D1 -61707 102000


In [314]:
verify_demand2 = verify_usage_demand(new_use_df)
verify_demand2['demand_check'] = [c-d for c, d in zip(verify_demand2['capacity'], verify_demand2['Demand (km)'])]
v_df2 = verify_demand2[verify_demand2['demand_check'] < 0]

fuel_adj = update_fuel_adj(verify_demand2)

print(100 - 70*(get_fuel_cost(new_fuel_plan) + get_veh_cost(new_purchase_plan))/ 172000000)
v_df2.head()


67.49802341904737


,Year,Size,Distance_bucket,capacity,Distance,Demand (km),demand_check


In [315]:
random.seed(192)
new_buy_df, new_sell_df = create_buy_sell_dfs(new_purchase_plan)
available = vehicle_running_total(new_buy_df, new_sell_df, new_use_df)
report, pp = available_check(available, new_purchase_plan)
report

need LNG_S1_2025
considering purchase_LNG_S1_2025_2029 2025 2029 2029
purchase_LNG_S1_2025_2029 3 1
need LNG_S4_2027
considering purchase_LNG_S4_2027_2030 2027 2030 2027
purchase_LNG_S4_2027_2030 3 2
considering purchase_LNG_S4_2027_2030 2027 2030 2030
purchase_LNG_S4_2027_2030 4 1
need BEV_S4_2028
considering purchase_BEV_S4_2028_2030 2028 2030 2029
purchase_BEV_S4_2028_2030 2 1
considering purchase_BEV_S4_2028_2030 2028 2030 2030
purchase_BEV_S4_2028_2030 3 1
need LNG_S4_2028
considering purchase_LNG_S4_2028_2031 2028 2031 2029
purchase_LNG_S4_2028_2031 2 1
need BEV_S3_2030
considering purchase_BEV_S3_2030_2034 2030 2034 2034
purchase_BEV_S3_2030_2034 6 1
need BEV_S4_2031
considering purchase_BEV_S4_2031_2034 2031 2034 2033
purchase_BEV_S4_2031_2034 2 1
need BEV_S4_2032
considering purchase_BEV_S4_2032_2035 2032 2035 2032
purchase_BEV_S4_2032_2035 3 1
need BEV_S2_2033
considering purchase_BEV_S2_2033_2039 2033 2039 2033
purchase_BEV_S2_2033_2039 3 1
need BEV_S4_2033
considering purch

,ID,Years,Amt
0,LNG_S1_2025,2029,1
1,LNG_S4_2027,2027,2
2,LNG_S4_2027,2030,1
3,BEV_S4_2028,2029,1
4,BEV_S4_2028,2030,1
5,LNG_S4_2028,2029,1
6,BEV_S3_2030,2034,1
7,BEV_S4_2031,2033,1
8,BEV_S4_2032,2032,1
9,BEV_S2_2033,2033,1


In [316]:
pct_of_inventory(new_buy_df, new_sell_df)

,Year,Num_Vehicles_x,Num_Vehicles_y,running_total,pct
0,2023,215,35,180,0.162791
1,2024,45,40,185,0.177778
2,2025,49,38,196,0.162393
3,2026,44,42,198,0.175000
4,2027,46,45,199,0.184426
5,2028,48,45,202,0.182186
6,2029,49,44,207,0.175299
7,2030,55,47,215,0.179389
8,2031,49,49,215,0.185606
9,2032,61,50,226,0.181159


In [317]:
random.seed(192)
new_buy_df2, new_sell_df2 = create_buy_sell_dfs(pp)
available2 = vehicle_running_total(new_buy_df2, new_sell_df2, new_use_df)
report, pp2 = available_check(available2, pp)
report

,ID,Years,Amt


In [261]:
submission = pd.concat([new_buy_df2, new_use_df, new_sell_df2])
submission = submission[submission['Num_Vehicles'] > 0]
submission['Year'] = submission['Year'].astype(int)
submission['Num_Vehicles'] = submission['Num_Vehicles'].astype(int)
submission['Distance_per_vehicle(km)'] = submission['Distance_per_vehicle(km)'].astype(float)


In [262]:
submission[['Year', 'ID', 'Num_Vehicles', 'Type', 'Fuel', 'Distance_bucket', 'Distance_per_vehicle(km)']].to_csv('solution56.csv', index=False)

0
Not Solved


In [324]:
for f in new_fuel_plan.keys():
    vname = re.sub('usage_', '', f)
    usage_variables[vname].setInitialValue(new_fuel_plan[f])
    
for p in pp.keys():
    vname = re.sub('purchase_', '', p)
    purchase_variables[vname].setInitialValue(pp[p])

In [327]:
print(total_cost.solve(PULP_CBC_CMD(timeLimit=14400, keepFiles=False, mip=True, warmStart=True)))
print(LpStatus[total_cost.status])

1
Optimal
